In [ ]:
#import libraries

import requests
import pandas as pd
import time
import credentials

In [ ]:
#api key imported from separate file

key = credentials.youtube

In [ ]:
channel_id = input('Channel Id: ')

In [ ]:
#extracting video statistics and returning values

def get_video_stats(video_id):
    url_stats = f'https://www.googleapis.com/youtube/v3/videos?key={key}&part=statistics&id={video_id}'
    stats_api = requests.get(url_stats).json()

    views = stats_api['items'][0]['statistics']['viewCount']
    likes = stats_api['items'][0]['statistics']['likeCount']
    comments = stats_api['items'][0]['statistics']['commentCount']

    return views, likes, comments

In [ ]:
#

def get_video(df): #passing dataframe
    pageToken = ''
    while 1:
        url_video = f'https://www.googleapis.com/youtube/v3/search?key={key}&channelId={channel_id}&part=snippet&type=video&order=date&maxResults=50&{pageToken}'
        video_api = requests.get(url_video).json()

        time.sleep(1.5)
        for video in video_api['items'] :
            channel_name = video['snippet']['channelTitle']
            video_title = video['snippet']['title']
            video_id = video['id']['videoId']
            
            views, likes, comments = get_video_stats(video_id)

            temp_df = pd.DataFrame({
            'channel_name' : [channel_name],
            'video_title' : [video_title],
            'video_id' : [video_id],
            'views' : [views],
            'likes' : [likes],
            'comments' : [comments]})

            df = pd.concat([df, temp_df], ignore_index=True)

        try:
            if video_api['nextPageToken'] != None:
                pageToken = 'pageToken=' + video_api['nextPageToken']
        except:
            break

    return df

In [ ]:
#creating dataframe to pass through functions

df = pd.DataFrame(columns=['channel_name','video_title','video_id','views','likes','comments','pos_sentiment','neutral','neg_sentiment'])

df2 = get_video(df)

In [ ]:
#grabbing youtube channel name to use in file name - replacing space with underscore

yt_channel = df2.channel_name[0].lower()
yt_channel = yt_channel.replace(' ', '_')

In [ ]:
df2.to_csv(f'c:/Users/Diego/Documents/{yt_channel}_data_stats.csv')

In [51]:
#importing natural language toolkit - sentiment analyzer

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [50]:
analysis = SentimentIntensityAnalyzer()

In [85]:
#grabbing a sample video id

test_id = df2.video_id[0]

In [86]:
#making api call to gather video comments

url_comments = f'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=10&videoId={test_id}&key={key}'
comment_request = requests.get(url_comments).json()

In [95]:
#variable assigned to random comment

test_comment = comment_request['items'][3]['snippet']['topLevelComment']['snippet']['textDisplay']

In [98]:
test_comment.replace("&#39;", "'")

"Nice video,  the balance test wasn't very accurate. Balance beads would fail this test."

In [99]:
#sentiment analysis

analysis.polarity_scores(test_comment)

{'neg': 0.191, 'neu': 0.656, 'pos': 0.153, 'compound': -0.1779}